In [1]:
import random
import math
from typing import List, Dict, Any

# ==============================================================================
# 1. CORE DATA MODELS
# These classes represent the fundamental objects in your game.
# ==============================================================================


class Booster:
    """Represents a single type of booster with its properties."""

    def __init__(self, name: str, price: int, effectiveness: float):
        """
        Args:
            name (str): The booster's name (e.g., "Time Warp").
            price (int): Cost in coins to purchase.
            effectiveness (float): A value from 0.0 to 1.0 representing how much
                                   this booster reduces level duration. 0.2 means 20% faster.
        """
        self.name = name
        self.price = price
        self.effectiveness = effectiveness

    def __repr__(self) -> str:
        return f"Booster(name='{self.name}')"


class Level:
    """Represents a single level in the game."""

    def __init__(self, level_id: int, base_duration: int, difficulty: float, rewards: Dict[str, Any]):
        """
        Args:
            level_id (int): The level number.
            base_duration (int): Expected time in seconds to complete for an average player.
            difficulty (float): A value from 0.0 to 1.0 indicating the level's challenge.
            rewards (Dict[str, Any]): Resources given upon completion.
                                      Example: {'coins': 50, 'boosters': {'Time Warp': 1}}
        """
        self.level_id = level_id
        self.base_duration = base_duration
        self.difficulty = difficulty
        self.rewards = rewards

    def __repr__(self) -> str:
        return f"Level(id={self.level_id}, difficulty={self.difficulty:.2f})"


class Player:
    """Represents a player archetype with their stats and inventory."""

    def __init__(self, player_id: str, skill_potential: float, booster_tendency: float, daily_playtime_budget: int, initial_inventory: Dict[str, Any]):
        """
        Args:
            player_id (str): A unique name for this player archetype (e.g., "Frugal_Expert").
            skill_potential (float): Innate skill (0.0 to 1.0). High skill negates difficulty.
            booster_tendency (float): Baseline willingness to use boosters (0.0 to 1.0).
            daily_playtime_budget (int): Total seconds the player will play in a day.
            initial_inventory (Dict[str, Any]): Starting resources.
        """
        self.player_id = player_id
        self.skill_potential = skill_potential
        self.booster_tendency = booster_tendency

        self.initial_playtime = daily_playtime_budget
        self.daily_playtime_budget = daily_playtime_budget

        # Deep copy inventory to prevent archetypes from sharing the same object
        self.inventory = {
            'coins': initial_inventory.get('coins', 0),
            'boosters': initial_inventory.get('boosters', {}).copy()
        }
        self.current_level = 0
        self.days_to_reach_target = None

    def get_total_boosters(self) -> int:
        """Returns the total count of all boosters in inventory."""
        return sum(self.inventory['boosters'].values())

    def has_boosters(self) -> bool:
        """Checks if the player has any boosters."""
        return self.get_total_boosters() > 0

    def consume_boosters(self, quantity: int, booster_definitions: Dict[str, Booster]):
        """
        Consumes a specified quantity of boosters from inventory.
        Picks randomly from available boosters. Returns the total effectiveness.
        """
        total_effectiveness = 0.0
        consumed_count = 0

        # Get a list of boosters the player actually owns
        owned_boosters = [
            b for b, count in self.inventory['boosters'].items() if count > 0]
        random.shuffle(owned_boosters)

        for booster_name in owned_boosters:
            if consumed_count >= quantity:
                break

            # Consume one of this booster
            self.inventory['boosters'][booster_name] -= 1
            total_effectiveness += booster_definitions[booster_name].effectiveness
            consumed_count += 1

        return total_effectiveness

    def add_rewards(self, rewards: Dict[str, Any]):
        """Adds level rewards to the player's inventory."""
        self.inventory['coins'] += rewards.get('coins', 0)
        booster_rewards = rewards.get('boosters', {})
        for name, count in booster_rewards.items():
            self.inventory['boosters'][name] = self.inventory['boosters'].get(
                name, 0) + count

    def reset_daily_playtime(self):
        """Resets the daily playtime budget to its initial value."""
        self.daily_playtime_budget = self.initial_playtime

    def __repr__(self) -> str:
        return f"Player(id='{self.player_id}', level={self.current_level}, coins={self.inventory['coins']}, boosters={self.get_total_boosters()})"

# ==============================================================================
# 2. THE SIMULATOR ENGINE
# This class orchestrates the entire simulation process.
# ==============================================================================


class GameSimulator:
    """Runs the game simulation for a set of players and levels."""

    # Tunable simulation parameters
    MAX_BOOSTERS_PER_LEVEL = 3

    def __init__(self, players: List[Player], levels: List[Level], boosters: List[Booster]):
        self.players = players
        self.levels = sorted(levels, key=lambda lvl: lvl.level_id)
        self.booster_definitions = {b.name: b for b in boosters}
        self.log = []

    def _simulate_level_attempt(self, player: Player, level: Level):
        """Core logic for a single player attempting a single level."""

        # --- a. Calculate Struggle ---
        struggle_score = max(0, level.difficulty - player.skill_potential)

        # --- b. Decide IF boosters will be used ---
        prob_to_use = player.booster_tendency + \
            (struggle_score * (1 - player.booster_tendency))

        boosters_to_use = 0
        booster_effectiveness = 0.0

        if player.has_boosters() and random.random() < prob_to_use:
            # --- c. Decide HOW MANY boosters will be used ---
            scores = []
            for k in range(1, self.MAX_BOOSTERS_PER_LEVEL + 1):
                score = (1 + struggle_score) ** k
                scores.append(score)

            # Weighted random choice to select quantity
            chosen_quantity = random.choices(
                population=range(1, self.MAX_BOOSTERS_PER_LEVEL + 1),
                weights=scores,
                k=1
            )[0]

            # Ensure player doesn't use more boosters than they own
            boosters_to_use = min(chosen_quantity, player.get_total_boosters())

        # --- d. Calculate Final Playtime ---
        base_time_cost = level.base_duration * (1 + struggle_score)

        if boosters_to_use > 0:
            booster_effectiveness = player.consume_boosters(
                boosters_to_use, self.booster_definitions)

        final_playtime = base_time_cost * (1 - booster_effectiveness)
        # Round up to nearest second
        final_playtime = math.ceil(final_playtime)

        return {
            "playtime": final_playtime,
            "boosters_used": boosters_to_use,
            "struggle_score": struggle_score
        }

    def _run_simulation_for_days(self, days_to_simulate: int):
        """Internal method to run simulation for a fixed number of days."""
        for day in range(1, days_to_simulate + 1):
            print(f"\n==================== DAY {day} ====================")
            for player in self.players:
                player.reset_daily_playtime()
                # ... rest of daily simulation logic ...
                # This is the same loop as before
                print(f"\n[SIMULATING] Player: {player.player_id}")
                print(
                    f"  > Start of Day State: Level {player.current_level}, Playtime Budget: {player.daily_playtime_budget}s, Coins: {player.inventory['coins']}, Boosters: {player.get_total_boosters()}")

                while player.daily_playtime_budget > 0 and player.current_level < len(self.levels):
                    current_level_obj = self.levels[player.current_level]
                    attempt_result = self._simulate_level_attempt(
                        player, current_level_obj)
                    playtime_cost = attempt_result["playtime"]

                    if player.daily_playtime_budget < playtime_cost:
                        break

                    player.daily_playtime_budget -= playtime_cost
                    player.add_rewards(current_level_obj.rewards)
                    player.current_level += 1

                    # ... logging ...
                    log_entry = {"day": day, "player_id": player.player_id, "level_id": current_level_obj.level_id, "struggle_score": f"{attempt_result['struggle_score']:.2f}", "boosters_used": attempt_result[
                        'boosters_used'], "playtime_cost": playtime_cost, "playtime_left": player.daily_playtime_budget, "final_coins": player.inventory['coins'], "final_boosters": player.get_total_boosters()}
                    self.log.append(log_entry)

                print(
                    f"  > End of Day State: Reached Level {player.current_level}, Playtime Left: {player.daily_playtime_budget}s, Coins: {player.inventory['coins']}, Boosters: {player.get_total_boosters()}")

    def _run_simulation_to_level(self, target_level: int):
        """Internal method to run simulation until a target level is reached."""
        day = 0
        while any(p.current_level < target_level for p in self.players):
            day += 1
            print(f"\n==================== DAY {day} ====================")

            if all(p.current_level >= len(self.levels) for p in self.players if p.current_level < target_level):
                print(
                    "\n[STOP] All players have run out of available levels before reaching the target.")
                break

            for player in self.players:
                if player.current_level >= target_level:
                    continue

                player.reset_daily_playtime()
                print(
                    f"\n[SIMULATING] Player: {player.player_id} (Target: Lv.{target_level})")

                while player.daily_playtime_budget > 0 and player.current_level < target_level and player.current_level < len(self.levels):
                    current_level_obj = self.levels[player.current_level]
                    attempt_result = self._simulate_level_attempt(
                        player, current_level_obj)
                    playtime_cost = attempt_result["playtime"]

                    if player.daily_playtime_budget < playtime_cost:
                        break

                    player.daily_playtime_budget -= playtime_cost
                    player.add_rewards(current_level_obj.rewards)
                    player.current_level += 1

                    # Record the day the player reached the target level
                    if player.current_level == target_level and player.days_to_reach_target is None:
                        player.days_to_reach_target = day

                    # ... logging ...
                    log_entry = {"day": day, "player_id": player.player_id, "level_id": current_level_obj.level_id, "struggle_score": f"{attempt_result['struggle_score']:.2f}", "boosters_used": attempt_result[
                        'boosters_used'], "playtime_cost": playtime_cost, "playtime_left": player.daily_playtime_budget, "final_coins": player.inventory['coins'], "final_boosters": player.get_total_boosters()}
                    self.log.append(log_entry)

    def run_simulation(self, target_days: int = None, target_level: int = None):
        """Runs the full simulation with a specified target."""
        if (target_days and target_level) or (not target_days and not target_level):
            raise ValueError(
                "Please specify either 'target_days' OR 'target_level'.")

        self.log = []
        print("--- Starting Game Simulation ---")

        if target_days:
            self._run_simulation_for_days(target_days)
        elif target_level:
            self._run_simulation_to_level(target_level)

        print("\n--- Simulation Complete ---")
        return self.log

    def print_summary(self):
        """Prints a formatted summary of the simulation log."""
        print("\n--- Simulation Log Summary ---")
        if not self.log:
            print("No logs recorded.")
            return

        for entry in self.log:
            print(
                f"--- Day {entry['day']} | Player: {entry['player_id']} | Level Cleared: {entry['level_id']} ---")
            print(f"  - Struggle Score: {entry['struggle_score']}")
            print(f"  - Boosters Used:  {entry['boosters_used']}")
            print(f"  - Time Cost:      {entry['playtime_cost']}s")
            print(f"  - State After Level:")
            print(f"    - Playtime Left: {entry['playtime_left']}s")
            print(f"    - Final Coins:   {entry['final_coins']}")
            print(f"    - Final Boosters: {entry['final_boosters']}")
            print("-" * 45)

    def print_player_information_summary(self):
        """Prints a detailed summary of each player's final state and stats."""
        print("\n--- Final Player Information Summary ---")
        for player in self.players:
            print(f"Player: {player.player_id}")
            print(f"  - Archetype Stats:")
            print(f"    - Skill Potential: {player.skill_potential}")
            print(f"    - Booster Tendency: {player.booster_tendency}")
            print(f"  - Final State:")
            print(f"    - Reached Level: {player.current_level}")
            if player.days_to_reach_target:
                print(
                    f"    - Days to Reach Target: {player.days_to_reach_target}")
            print(f"    - Final Coins:   {player.inventory['coins']}")
            print(f"    - Final Boosters:")
            if not player.inventory['boosters'] or all(v == 0 for v in player.inventory['boosters'].values()):
                print("      - None")
            else:
                for name, count in player.inventory['boosters'].items():
                    if count > 0:
                        print(f"      - {name}: {count}")
            print("-" * 30)


# ==============================================================================
# 3. SIMULATION SETUP & EXECUTION
# This is where you define your game's data and run the simulator.
# ==============================================================================

if __name__ == "__main__":
    # --- Define Boosters ---
    booster_list = [
        Booster(name="Speedy Time", price=50,
                effectiveness=0.20),  # 20% time reduction
        Booster(name="Mega Clear", price=75,
                effectiveness=0.35)    # 35% time reduction
    ]

    # --- Define Levels (Create a progression curve) ---
    level_list = []
    # Create 100 levels for a longer simulation
    for i in range(1, 101):
        rewards = {'coins': 10 + i * 2}
        if i % 4 == 0:
            rewards['boosters'] = {random.choice(booster_list).name: 1}

        level_list.append(Level(
            level_id=i,
            base_duration=60 + i * 5,
            # Difficulty ramps from ~0.1 to ~0.9
            difficulty=0.1 + (i / 100) * 0.8,
            rewards=rewards
        ))

    default_init_inventory = {
        'coins': 100,
        'boosters': {
            'Speedy Time': 3,
            'Mega Clear': 1
        }
    }

    # --- Define Player Archetypes ---
    player_archetypes = [
        Player(
            player_id="Frugal_Expert",
            skill_potential=0.8,
            booster_tendency=0.1,
            daily_playtime_budget=1800,  # 30 minutes
            initial_inventory=default_init_inventory.copy()
        ),
        Player(
            player_id="Average_Joe",
            skill_potential=0.5,
            booster_tendency=0.5,
            daily_playtime_budget=1800,  # 30 minutes
            # initial_inventory={'coins': 100, 'boosters': {'Speedy Time': 3, 'Mega Clear': 1}}
            initial_inventory=default_init_inventory.copy()
        ),
        Player(
            player_id="Rich_Spender",
            skill_potential=0.2,
            booster_tendency=0.9,
            daily_playtime_budget=1800,  # 30 minutes
            initial_inventory=default_init_inventory.copy()
        )
    ]

    # --- Run the Simulation to answer the question about Level 54 ---
    # NOTE: The simulation will stop once all players have PASSED level 53 (i.e., reached level 54)
    TARGET_LEVEL = 54
    simulator = GameSimulator(player_archetypes, level_list, booster_list)
    simulation_log = simulator.run_simulation(target_level=TARGET_LEVEL)

    # --- Print the Results ---
    # The full log is too long, so we will just print the final summary
    # simulator.print_summary() # Uncomment if you want to see the full level-by-level log

    simulator.print_player_information_summary()
    simulator.print_summary()

--- Starting Game Simulation ---

==================== DAY 1 ====================

[SIMULATING] Player: Frugal_Expert (Target: Lv.54)

[SIMULATING] Player: Average_Joe (Target: Lv.54)

[SIMULATING] Player: Rich_Spender (Target: Lv.54)

==================== DAY 2 ====================

[SIMULATING] Player: Frugal_Expert (Target: Lv.54)

[SIMULATING] Player: Average_Joe (Target: Lv.54)

[SIMULATING] Player: Rich_Spender (Target: Lv.54)

==================== DAY 3 ====================

[SIMULATING] Player: Frugal_Expert (Target: Lv.54)

[SIMULATING] Player: Average_Joe (Target: Lv.54)

[SIMULATING] Player: Rich_Spender (Target: Lv.54)

==================== DAY 4 ====================

[SIMULATING] Player: Frugal_Expert (Target: Lv.54)

[SIMULATING] Player: Average_Joe (Target: Lv.54)

[SIMULATING] Player: Rich_Spender (Target: Lv.54)

==================== DAY 5 ====================

[SIMULATING] Player: Frugal_Expert (Target: Lv.54)

[SIMULATING] Player: Average_Joe (Target: Lv.54)

[SIMULA

In [2]:
for level in level_list:
    print(f"{level.base_duration}\t{level.difficulty}")

65	0.10800000000000001
70	0.116
75	0.124
80	0.132
85	0.14
90	0.14800000000000002
95	0.15600000000000003
100	0.164
105	0.172
110	0.18000000000000002
115	0.188
120	0.196
125	0.20400000000000001
130	0.21200000000000002
135	0.22
140	0.228
145	0.23600000000000002
150	0.244
155	0.252
160	0.26
165	0.268
170	0.276
175	0.28400000000000003
180	0.29200000000000004
185	0.30000000000000004
190	0.30800000000000005
195	0.31600000000000006
200	0.32400000000000007
205	0.33199999999999996
210	0.33999999999999997
215	0.348
220	0.356
225	0.364
230	0.372
235	0.38
240	0.388
245	0.396
250	0.404
255	0.41200000000000003
260	0.42000000000000004
265	0.42800000000000005
270	0.43600000000000005
275	0.44400000000000006
280	0.45200000000000007
285	0.4600000000000001
290	0.4680000000000001
295	0.476
300	0.484
305	0.492
310	0.5
315	0.508
320	0.516
325	0.524
330	0.532
335	0.54
340	0.548
345	0.5559999999999999
350	0.564
355	0.572
360	0.58
365	0.588
370	0.596
375	0.604
380	0.612
385	0.62
390	0.628
395	0.636
400	0.644
405